In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
sys.dont_write_bytecode
import os
import json
import time
# TOPOLOGIES
from network_topologies.pseudogeant import Pseudogeant
from network_topologies.geant import Geant
from network_topologies.topology import Topology
from network_topologies.topology_preprocessing import preprocess_metadata
# ROUTING ALGORITHMS
from routing_algorithms.dijkstra import dijkstra
from routing_algorithms.dijkstra import calculate_path
from routing_algorithms.dijkstra import set_spt
from routing_algorithms.ear import ear
from routing_algorithms.mora import *
from routing_algorithms.mora_v2 import *
# NETWORK OBJECTS
from utils.network_objects import *
# PA SERVICES
from service_flows.traffic_generator import TrafficGenerator
from itertools import permutations
import random
import numpy as np
from deap import algorithms, base, creator, tools, algorithms
import collections
import matplotlib.pyplot as plt
import import_ipynb
import operator
import pandas as pd

In [7]:
np.random.seed(64)


In [8]:
topo = Geant(routing_method='EAR')
topo.save_topology_info()

Resetting network topology... OK!


In [9]:
# INSTANTIATE TRAFFIC GENERATOR
path = '/Users/federicocolangelo/Documents/Codici/MORA-Multi-Objective-Routing-Algorithm/service_flows/experiments_1/'
set_spt(topo)
tg = TrafficGenerator(interval=60, topology=topo, path=path, faults = 0, traffic_boost = 0)
time.sleep(1)


******* GENERATE_FLOWS -> ITERATION 1 OUT OF 288 *******
++++ BEGINNING OF ITERATION @ 04/29/2020, 16:24:22 ++++

#### NODE FAILURES PHASE... execution time = 4.0531158447265625e-06
#### DISRUPTED FLOWS REROUTING PHASE... execution time = 5.888938903808594e-05
#### GENERATE NEW FLOWS PHASE... execution time = 7.0508270263671875
++++ END OF ITERATION @ 04/29/2020, 16:24:29 ++++
******* GENERATE_FLOWS -> ITERATION 1 OUT OF 288 ELAPSED TIME = 7.075161933898926 *******
--------------------------------------------------------------------------

******* GENERATE_FLOWS -> ITERATION 2 OUT OF 288 *******
++++ BEGINNING OF ITERATION @ 04/29/2020, 16:25:22 ++++

#### NODE FAILURES PHASE... execution time = 4.0531158447265625e-06
#### DISRUPTED FLOWS REROUTING PHASE... execution time = 4.7206878662109375e-05
#### GENERATE NEW FLOWS PHASE... execution time = 29.379831075668335
++++ END OF ITERATION @ 04/29/2020, 16:25:51 ++++
******* GENERATE_FLOWS -> ITERATION 2 OUT OF 288 ELAPSED TIME = 29.386986

In [66]:
%timeit mutate_path(indi[5], topo, list)

48.5 ms ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
--------------------------------------------------------------------------

******* GENERATE_FLOWS -> ITERATION 2 OUT OF 12 *******
++++ BEGINNING OF ITERATION @ 04/15/2020, 12:10:38 ++++

#### NODE FAILURES PHASE... execution time = 1.0967254638671875e-05
#### DISRUPTED FLOWS REROUTING PHASE... execution time = 3.981590270996094e-05
#### GENERATE NEW FLOWS PHASE... execution time = 

In [72]:
%timeit mutate_path_faster(indi[5], topo, list)

662 µs ± 64.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
--------------------------------------------------------------------------

******* GENERATE_FLOWS -> ITERATION 8 OUT OF 12 *******
++++ BEGINNING OF ITERATION @ 04/15/2020, 12:16:38 ++++

#### NODE FAILURES PHASE... execution time = 9.059906005859375e-06
#### DISRUPTED FLOWS REROUTING PHASE... execution time = 3.910064697265625e-05
#### GENERATE NEW FLOWS PHASE... execution time = 18.27043604850769
++++ END OF ITERATION @ 04/15/2020, 12:16:56 ++++
******* GENERATE_FLOWS -> ITERATION 8 OUT OF 12 ELAPSED TIME = 18.286648988723755 *******


In [74]:
topology=topo
flow_obj = Flow(test)
# number of weights in the tuple -> number of objective functions
creator.create("FitnessMultiObj", base.Fitness, weights=(-1.0, -1.0, -1.0, -1.0,)) 
creator.create("Individual", list, fitness=creator.FitnessMultiObj)
toolbox = base.Toolbox()
toolbox.register("individual", generate_individual, creator.Individual, flow_obj.starting_node, flow_obj.ending_node, topology)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", crossover_one_point, topology=topology, ind_class=creator.Individual, toolbox=toolbox)
toolbox.register("select", tools.selNSGA2)
toolbox.register("mutate", mutate_path, topology=topology, indi_class=creator.Individual)

evaluate_individual = get_evaluate_individual(topology, test)
toolbox.register("evaluate", evaluate_individual)
evaluate_SLA = get_evaluate_SLA(flow_obj.SLA, topology, evaluate_individual)
penalty = get_penalty(flow_obj.SLA, topology, evaluate_individual) 
toolbox.decorate("evaluate", tools.DeltaPenality(evaluate_SLA, 20.0, penalty))

In [17]:
from routing_algorithms.mora import get_faster_optimize_route
faster_optimize_flow = get_faster_optimize_route(topo)

In [18]:
# Pop = 1000 -> 30 s di media (LOL)
# Pop = 100 -> 1.5 s
# Pop = 50 -> 730 ms
# Pop = 25 -> 370 ms
%timeit faster_optimize_flow(test)

731 ms ± 107 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# Pop = 25
%timeit faster_optimize_flow(test)

In [86]:
%timeit faster_optimize_flow(test)

671 ms ± 162 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [81]:
%timeit optimize_flow(test)

5.12 s ± 940 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit crossover_one_point(indi[5], indi[6], topo, list, toolbox)

In [ ]:
%timeit evaluate_individual(indi[5])

In [132]:
def enumerate_paths(s, d, max_hops, current_prefix = [], pts = []):
    current_prefix.append(s)
    N = topo.get_valid_links(current_prefix[-1])
    N = [x for x in N if x not in current_prefix]
    for n in N:
        if d == n:
            new = current_prefix.copy()
            new.append(d)
            pts.append(new)
            continue
        if len(current_prefix) <= max_hops-1:
            pts = enumerate_paths(n, d, max_hops, current_prefix, pts)
    current_prefix.pop(-1)
    return pts         

In [42]:
f = {'node1': 'UK', 'node2': 'IR', 'loss': 0, 'latency': 150, 'jitter': 0, 'bandwidth': 412.48, '_id': 'ESPLpremium'}
evaluation = get_evaluate_individual(topo, f)
shortest = topo.get_shortest_path(f)
shortest_eval = evaluation(shortest)
shorts = topo.enumerate_paths('IT', 'IR', len(shortest)+2)

shorts_eval = []
for p in shorts:
    e = evaluation(p)
    comp = []
    for idx in range(1, len(e)):
        if shortest[idx] > e[idx]:
            


['IT', 'CH', 'FR', 'UI', 'IR'] 

21
[['IT', 'CH', 'DE', 'NL', 'UK', 'IE', 'IR'], ['IT', 'CH', 'DE', 'NL', 'UK', 'UI', 'IR'], ['IT', 'CH', 'ES', 'FR', 'UI', 'IR'], ['IT', 'CH', 'FR', 'UI', 'IR'], ['IT', 'CH', 'FR', 'UI', 'UK', 'IE', 'IR'], ['IT', 'FN', 'CH', 'ES', 'FR', 'UI', 'IR'], ['IT', 'FN', 'CH', 'FR', 'UI', 'IR'], ['IT', 'CH', 'DE', 'NL', 'UK', 'IE', 'IR'], ['IT', 'CH', 'DE', 'NL', 'UK', 'UI', 'IR'], ['IT', 'CH', 'ES', 'FR', 'UI', 'IR'], ['IT', 'CH', 'FR', 'UI', 'IR'], ['IT', 'CH', 'FR', 'UI', 'UK', 'IE', 'IR'], ['IT', 'FN', 'CH', 'ES', 'FR', 'UI', 'IR'], ['IT', 'FN', 'CH', 'FR', 'UI', 'IR'], ['IT', 'CH', 'DE', 'NL', 'UK', 'IE', 'IR'], ['IT', 'CH', 'DE', 'NL', 'UK', 'UI', 'IR'], ['IT', 'CH', 'ES', 'FR', 'UI', 'IR'], ['IT', 'CH', 'FR', 'UI', 'IR'], ['IT', 'CH', 'FR', 'UI', 'UK', 'IE', 'IR'], ['IT', 'FN', 'CH', 'ES', 'FR', 'UI', 'IR'], ['IT', 'FN', 'CH', 'FR', 'UI', 'IR'], ['IT', 'CH', 'DE', 'NL', 'UK', 'IE', 'IR'], ['IT', 'CH', 'DE', 'NL', 'UK', 'UI', 'IR'], ['IT', 'CH', 'ES', 'FR'

In [ ]:
f = {'node1': 'UK', 'node2': 'IR'}
topo

In [35]:
len_list = []
for ni in topo.node_names:
    for nj in topo.node_names:
        if ni != nj:
            f = {'node1': ni, 'node2': nj}
            shortest = topo.get_shortest_path(f)
            pt = topo.enumerate_paths(ni, nj, len(shortest)+2, [], [])
            print('Path from {} to {}: shortest path has length {}, adding 2 hops lead to {} possible paths'.format(ni, nj, len(shortest), len(pt)))
            len_list.append(len(pt))

Path from SI to BE: shortest path has length 6, adding 2 hops lead to 73 possible paths
Path from SI to CZ: shortest path has length 4, adding 2 hops lead to 20 possible paths
Path from SI to HR: shortest path has length 4, adding 2 hops lead to 16 possible paths
Path from SI to PL: shortest path has length 4, adding 2 hops lead to 17 possible paths
Path from SI to ES: shortest path has length 4, adding 2 hops lead to 16 possible paths
Path from SI to DE: shortest path has length 4, adding 2 hops lead to 24 possible paths
Path from SI to CH: shortest path has length 3, adding 2 hops lead to 12 possible paths
Path from SI to LN: shortest path has length 6, adding 2 hops lead to 19 possible paths
Path from SI to SK: shortest path has length 4, adding 2 hops lead to 16 possible paths
Path from SI to UI: shortest path has length 5, adding 2 hops lead to 28 possible paths
Path from SI to IR: shortest path has length 6, adding 2 hops lead to 43 possible paths
Path from SI to GC: shortest pat

In [37]:
np.mean(len_list)

26.2752688172043

In [28]:
for ni in topo.node_names:
    for nj in topo.node_names:
        if ni != nj:
            pt = topo.enumerate_paths(ni, nj, 31, [], [])
            print(ni, nj, len(pt))


IT IR 984
IT CZ 689
IT GR 494
IT HU 921
IT LV 682
IT AT 494
IT GC 494
IT DE 314
IT NL 461
IT LT 682
IT PR 1054
IT SI 921
IT PT 1054
IT BE 592
IT FN 914
IT EE 682
IT DU 538
IT ES 978
IT UK 406
IT CH 914
IT IE 984
IT HR 1162
IT UI 908
IT RO 1162
IT BG 1162
IT ET 682
IT FR 832
IT PL 584
IT LN 682
IT SK 1162
IR IT 984
IR CZ 1564
IR GR 1376
IR HU 1320
IR LV 1448
IR AT 692
IR GC 1376
IR DE 748
IR NL 1256
IR LT 1448
IR PR 1878
IR SI 1624
IR PT 1878
IR BE 1568
IR FN 1434
IR EE 1448
IR DU 1300
IR ES 1566
IR UK 628
IR CH 944
IR IE 628
IR HR 1736
IR UI 628
IR RO 1736
IR BG 1736
IR ET 1448
IR FR 1254
IR PL 1224
IR LN 1448
IR SK 1736
CZ IT 689
CZ IR 1564
CZ GR 1162
CZ HU 865
CZ LV 2094
CZ AT 568
CZ GC 1162
CZ DE 1216
CZ NL 961
CZ LT 2094
CZ PR 1450
CZ SI 1281
CZ PT 1450
CZ BE 1208
CZ FN 824
CZ EE 2094
CZ DU 1590
CZ ES 1274
CZ UK 794
CZ CH 460
CZ IE 1564
CZ HR 1194
CZ UI 1388
CZ RO 1194
CZ BG 1194
CZ ET 2094
CZ FR 1212
CZ PL 1852
CZ LN 2094
CZ SK 1194
GR IT 494
GR IR 1376
GR CZ 1162
GR HU 1172
GR LV

KeyboardInterrupt: 

In [48]:
from routing_algorithms.mora_v2 import get_optimize_route

get_path = get_optimize_route(topo, toolbox)

NameError: name 'toolbox' is not defined

In [40]:
a = []
b = ['1', '2']
if a:
    print(a)
if b:
    print(b)

['1', '2']
